# Data loading

In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import metrics
import lightgbm as lgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [6]:
import os
import boto3
from dotenv import load_dotenv
import io 
import pandas as pd

load_dotenv()

YC_ACCESS_KEY_ID = os.getenv("YC_ACCESS_KEY_ID")
YC_SECRET_ACCESS_KEY = os.getenv("YC_SECRET_ACCESS_KEY")
YC_ENDPOINT_URL = os.getenv("YC_ENDPOINT_URL")
YC_BUCKET_NAME = os.getenv("YC_BUCKET_NAME")

In [7]:
session = boto3.session.Session()
s3_client = session.client(
    service_name='s3',
    endpoint_url=YC_ENDPOINT_URL,
    aws_access_key_id=YC_ACCESS_KEY_ID,
    aws_secret_access_key=YC_SECRET_ACCESS_KEY
)

In [8]:
import gzip

file_names = ["item_categories.csv.gzip", "items.csv.gzip", "sample_submission.csv.gzip", "shops.csv.gzip", "test.csv.gzip", "train.csv.gzip"]
data_location = "filtered_data/"

data_storage = dict()
for file_name in file_names:
    response = s3_client.get_object(Bucket=YC_BUCKET_NAME, Key=f'{data_location}{file_name}')
    data_storage[file_name[: len(file_name) - 9]] = pd.read_csv(io.BytesIO(response['Body'].read()), compression='gzip')

data_storage["train"]['date'] = pd.to_datetime(data_storage["train"]['date'], format = '%Y-%m-%d')



# Data preporation tools

In [10]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from src.data_preprocessor import DataPreprocessor

proc = DataPreprocessor(data_storage)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
sales           = data_storage['train']
items           = data_storage['items']
item_categories = data_storage['item_categories']
shops           = data_storage['shops']

In [12]:
def create_testlike_train(df):
    matrix = []
    min_date = df['date'].min()
    for i in range(df['date_block_num'].min(), df['date_block_num'].max()+1):
        shops = df[df['date_block_num'] == i]['shop_id'].unique()
        items = df[df['date_block_num'] == i]['item_id'].unique()
        month_start = min_date + pd.tseries.offsets.DateOffset(months = i)
        matrix.append( np.array( list(product([i],[month_start],shops,items))))
        
    df_new = pd.DataFrame(np.vstack(matrix),columns = ['date_block_num','month_start','shop_id','item_id'])
    pivot = pd.pivot_table(df, 
                            values = ['item_cnt_day'], 
                            index = ['date_block_num','shop_id','item_id'], 
                            aggfunc = 'sum').reset_index()
    pivot2 = pd.pivot_table(df[df['item_cnt_day']>0], 
                            values = ['item_cnt_day'], 
                            index = ['date_block_num','shop_id','item_id'], 
                            aggfunc = 'count').reset_index()
    pivot2.rename(columns={'item_cnt_day': 'purch_cnt_month'}, inplace=True)
    
    df_new = df_new.merge(right = pivot, how = 'left', on = ['date_block_num','shop_id','item_id'], sort = False)
    df_new = df_new.merge(right = pivot2, how = 'left', on = ['date_block_num','shop_id','item_id'], sort = False)
    
    df_new.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
    df_new['item_cnt_month'] = df_new['item_cnt_month'].clip(0,20)
    df_new['item_cnt_month'].fillna(0, inplace=True)

    df_new['date_block_num'] = df_new['date_block_num'].astype('int8')
    df_new['shop_id'] = df_new['shop_id'].astype('int8')
    df_new['item_id'] = df_new['item_id'].astype('int16')
    df_new['item_cnt_month'] = df_new['item_cnt_month'].astype('float32')
    return df_new

In [13]:
%%time
df = create_testlike_train(sales)

/tmp/ipykernel_107464/3262818551.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new['item_cnt_month'].fillna(0, inplace=True)


CPU times: user 16.5 s, sys: 2.02 s, total: 18.5 s
Wall time: 18.5 s


In [14]:
df = proc.add_month_and_days(df)

df = proc.add_cat_features(df)

cat_features = ['shop_city', 'shop_type', 'item_category_type', 'item_category_subtype']
df = proc.encode_categorical_features(df, cat_features=cat_features)


In [15]:
columns_to_exclude = [
                      'item_cnt_month',
                      'purch_cnt_month',
                      'month_start',
                      'shop_name',
                      'item_category_name',
                      'item_name'
                     ]
cat_features = ['month',
                'shop_id',
                'shop_city',
                'shop_type',
                'item_category_id',
                'item_category_type',
                'item_category_subtype',
                'days_in_month'
               ]

In [16]:
# modeling
params = {'metric': 'rmse',
          'objective': 'mse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

# Prepare training and validation datasets with categorical features
train_data = lgb.Dataset(
    df[(df['date_block_num'] >= 19) & (df['date_block_num'] < 33)].drop(columns_to_exclude, axis=1),
    label=df[(df['date_block_num'] >= 19) & (df['date_block_num'] < 33)]['item_cnt_month'],
    categorical_feature=cat_features
)

valid_data = lgb.Dataset(
    df[df['date_block_num'] == 33].drop(columns_to_exclude, axis=1),
    label=df[df['date_block_num'] == 33]['item_cnt_month'],
    categorical_feature=cat_features,
    reference=train_data
)

# Validation schema

In [ ]:
from src.validation_schema import TimeSeriesRollingValidator

validator = TimeSeriesRollingValidator(df, 'date_block_num', train_window=8, test_window=1)
splits = validator.split_data_rolling()

In [32]:
# Train model
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    num_boost_round=1500,
    valid_sets=[train_data, valid_data],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
)

[LightGBM] [Info] Total Bins 518
[LightGBM] [Info] Number of data points in the train set: 3191064, number of used features: 10
[LightGBM] [Info] Start training from score 0.295206
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.05694	valid_1's rmse: 0.921393


KeyboardInterrupt: 